# Primeiro contato

Explorando a API percebemos que ela possui um limite muito baixo de uso:

* 500 requisições por hora
* 4 requisições por segundo

Para evitar que o nossa aplicação fique fora do ar por motivos de alta demanda de uso, além de tornar o carregamento mais rápido, vamos armazenar os dados localmente. Utilizaremos como base a imagem provida pelo [Ergast](https://ergast.com/mrd/) que está descontinuado e para carregar novos dados iremos utilizar a API do [Jolpica](https://github.com/jolpica/jolpica-f1).

Aqui faremos uma análise exploratória utilizando os dados do eargast baixando a imagem da base de dados até 2024(ano em que foi descontinuado).

In [5]:
from utils.ergast import download_ergast

download_ergast()

Data already downloaded.


## Os dados

In [6]:
import pandas as pd
from utils.ergast import load_ergast
import plotly.express as px

data = load_ergast()

drivers = data['drivers']
results = data['results']
driver_standings = data['driver_standings']
constructors = data['constructors']
constructor_results = data['constructor_results']
constructor_standings = data['constructor_standings']
races = data['races']
circuits = data['circuits']
lap_times = data['lap_times']
pit_stops = data['pit_stops']
qualifying = data['qualifying']
seasons = data['seasons']
sprint_results = data['sprint_results']
status = data['status']

In [7]:
fig = px.bar(
    races.groupby('year').count().reset_index(),
    x='year',
    y='raceId',
)

fig.update_layout(
    title='Total de corridas por ano',
    xaxis_title='Ano',
    yaxis_title='Número de corridas',
)

Podemos ver que o numero de corridas cresce drasticamente ao longo dos anos. Será que o número de pilotos segue a mesma lógica?

In [8]:
fig = px.bar(
    pd.merge(
        races[['raceId', 'year']],
        driver_standings[['raceId', 'driverId', 'points']],
        how='inner',
    )[['year', 'driverId']].drop_duplicates().year.value_counts().sort_index().reset_index(),
    x='year',
    y='count'
)

fig.update_layout(
    title='Total de pilotos por ano',
    xaxis_title='Ano',
    yaxis_title='Número de pilotos',
)

In [9]:

fig = px.bar(
    pd.merge(
        races[['raceId', 'year']],
        driver_standings[['raceId', 'driverId']],
        how='inner',
    ).groupby(['year', 'raceId']).count().groupby('year').driverId.mean().reset_index(),
    x='year',
    y='driverId'
)

fig.update_layout(
    title='Média de pilotos por corrida',
    xaxis_title='Ano',
    yaxis_title='Média de pilotos por corrida',
)

Interessante como mesmo observando a média de pilotos por corrida esse número ainda permanece elevado no início das corridas e como esse número tem se mantido constante nas últimas décadas.

### As tabelas

Dentro dos dados utilizados temos muitos campos que poderiam ser utilizados, desde o tempo exato de cada volta de cada piloto em cada corrida, até informações específicas sobre o clima em cada corrida.

Contudo, na fórmula 1, principalmente no início de temporada, é comum dizer que os resultados ainda "não querem dizer nada" ou "só dá para ter certeza depois da primeira metade". Pensamos em averiguar a veracidade dessas falas, será que realmente a classificação dos pilotos muda tanto com relação à primeira corrida ou desde o começo da temporada já podemos dizer com boa precisão quem serão os possíveis ganhadores?